<a href="https://colab.research.google.com/github/alkarps/GB.Methods-for-collecting-and-processing-data-from-the-Internet/blob/hw02/hw02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#!pip install requests
#!pip install lxml

In [2]:
from pprint import pprint
from lxml import html
import requests

In [3]:
class New:
    def __init__(self, source, title, link, date):
        self.source = source
        self.title = title
        self.link = link
        self.date = date
    def __str__(self):
        return f"source={self.source}; title={self.title}; link={self.link}; date={self.date}"

Написать приложение или функцию, которые собирают основные новости с сайтов на выбор lenta.ru, yandex-новости. Для парсинга использовать XPath.

Структура данных должны содержать:
 - *название источника;
 - наименование новости;
 - ссылку на новость;
 - дата публикации.


 Минимум один сайт, максимум - все два.

In [4]:
lenta_url = "https://lenta.ru"
lenta_uri_news = "//a[contains(@class, '_topnews') or contains(@class, '_longgrid')  or contains(@class, 'card-feature') ]"
lenta_root_classes = ("card-mini", "card-feature", "card-big")

In [5]:
res = requests.get(lenta_url)

In [6]:
dom = html.fromstring(res.text)

In [7]:
news = dom.xpath(lenta_uri_news)

In [8]:
def parseNewForLenta(new):
  link =  new.attrib["href"]
  if link.startswith("/"):
    link = lenta_url + link
  source = link.split("/")[2]
  date = ""
  title = ""
  current_root_class = None
  for root_class in lenta_root_classes:
    if root_class in new.classes:
      current_root_class = root_class
      break
  sub_title = ""
  for ch in new.getiterator():
    if f"{current_root_class}__date" in ch.classes:
      date = ch.text
    elif f"{current_root_class}__title" in ch.classes:
      main_title = ch.text
    elif f"{current_root_class}__rightcol" in ch.classes:
      sub_title = ch.text
  divider = ":"
  if main_title.endswith((".","?","!")):
    divider = " "
  if sub_title == "":
    divider = ""
  title = f"{main_title}{divider}{sub_title}"
  return New(source, title, link, date)

In [9]:
result = list()
for new in news:
  _new = parseNewForLenta(new)
  result.append(_new)